In [ ]:
!pip install statsmodels

In [ ]:
import math
import random
import re

import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats as st
from scipy.stats import norm

# Огляд scipy.stats

In [ ]:
rv = norm()

In [ ]:
rv

In [ ]:
# dir(rv)

In [ ]:
st.rv_continuous

In [ ]:
dist_continu = [d for d in dir(st) if
                isinstance(getattr(st, d), st.rv_continuous)]
dist_discrete = [d for d in dir(st) if
                 isinstance(getattr(st, d), st.rv_discrete)]
print('Кількість неперервних розподілів: %d' % len(dist_continu))
print('Кількість дискретних розподілів:   %d' % len(dist_discrete))

In [ ]:
norm.cdf(0)

Можемо вивести для декількох точок.

In [ ]:
norm.cdf([-1., 0, 1])

In [ ]:
norm.cdf(2, 0, 1)

Якщо хочемо отримати CDF і PDF певного розподілу:

In [ ]:
x = np.linspace(-5, 5, 5000)
mu = 0
sigma = 1

y_pdf = norm.pdf(x, mu, sigma)  # pdf нормального розподілу
y_cdf = norm.cdf(x, mu, sigma)  # cdf нормального розподілу

plt.plot(x, y_pdf, label='pdf')
plt.plot(x, y_cdf, label='cdf')
plt.legend();

Можемо дослідити, як параметри розподілу впливають на його вигляд:

In [ ]:
def plot_normal(x_range, mu=0, sigma=1, cdf=False, **kwargs):
    x = x_range
    if cdf:
        y = norm.cdf(x, mu, sigma)
    else:
        y = norm.pdf(x, mu, sigma)
    plt.plot(x, y, **kwargs)

In [ ]:
plot_normal(x, -2, 1, color='red', lw=2, ls='-', alpha=0.5)
plot_normal(x, 2, 1.2, color='blue', lw=2, ls='-', alpha=0.5)
plot_normal(x, 0, 0.8, color='green', lw=2, ls='-', alpha=0.5);

Створимо та проведемо дослідження вибірки із заданого розподілу:

In [ ]:
from scipy.stats import beta

# для відтворюваності
np.random.seed(seed=233423)

a, b = 2, 6

x = beta.rvs(a, b, size=1000)

In [ ]:
x.mean()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(x, density=True, alpha=0.2)
plt.show()

Виведемо описові статистики розподілу:

In [ ]:
m = beta.mean(a,b)
v = beta.var(a,b)
shp_a = beta.a
shp_b = beta.b
median = beta.median(a,b)

rv_stats = {"mean": m, "var": v, "shape a": shp_a, "shape b": shp_b, "median": median}
_ = [print(k,":",f'{v:.3f}') for k,v in rv_stats.items()]

Перші 4 моменти (про [моменти](https://www.analyticsvidhya.com/blog/2022/01/moments-a-must-known-statistical-concept-for-data-science/#:~:text=%E2%80%93%20Standardized%20Moments-,What%20is%20the%20Moment%20in%20Statistics%3F,as%20the%20X's%20expected%20values.)) :

In [ ]:
moments_values = beta.stats(a,b, moments="mvsk")

moments_names = ["mean", "var", "skew", "kurt"]
moments = dict(zip(moments_names, moments_values))
_ = [print(k,":",f'{v:.3f}') for k,v in moments.items()]

# Узгодження розподілу з даними

Припустимо, ми хочемо зʼясувати, за яким законом розподілу розподілені наші дані. Для цього можемо спробувати зафітити кілька різних розподілів і подивитися, який найкраще описує наші дані.

In [ ]:
def doane_formula(data):
    """
    Формула для пошуку оптимальної кількості бінів для візуалізації та аналізу даних:
    https://en.wikipedia.org/wiki/Histogram#Doane's_formula
    """
    N = len(data)
    skewness = st.skew(data)
    sigma_g1 = math.sqrt((6*(N-2))/((N+1)*(N+3)))
    num_bins = 1 + math.log(N,2) + math.log(1+abs(skewness)/sigma_g1,2)
    num_bins = round(num_bins)
    return num_bins

def plot_histogram(data, results, n):
    # N перших розподілів у ранжуванні
    N_DISTRIBUTIONS = {k: results[k] for k in list(results)[:n]}

    # Гістограма даних
    plt.figure(figsize=(10, 5))
    plt.hist(data, density=True, ec='white', color=(63/235, 149/235, 170/235))
    plt.title('HISTOGRAM')
    plt.xlabel('Values')
    plt.ylabel('Frequencies')
    
    # Розподіл
    for distribution, result in N_DISTRIBUTIONS.items():
        sse = result[0]
        arg = result[1]
        loc = result[2]
        scale = result[3]
        x_plot = np.linspace(min(data), max(data), 1000)
        y_plot = distribution.pdf(x_plot, loc=loc, scale=scale, *arg)
        dist_name = re.search(r'_continuous_distns\.(.*?) object', str(distribution)).group(1)
        plt.plot(x_plot, y_plot, label=dist_name + ": " + str(sse)[0:6], color=(random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)))
    
    plt.legend(title='DISTRIBUTIONS', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

def fit_data(data):
    ALL_DISTRIBUTIONS = [d for d in dir(st) if
                isinstance(getattr(st, d), st.rv_continuous)]
    
    MY_DISTRIBUTIONS = [st.beta, st.expon, st.norm, 
                        st.uniform, st.johnsonsb, st.gennorm,
                        st.gausshyper]

    # Шукаємо оптимальну кількість бінів у гістограмі за формулою Доані
    num_bins = doane_formula(data)
    frequencies, bin_edges = np.histogram(data, num_bins, density=True)
    central_values = [(bin_edges[i] + bin_edges[i+1])/2 for i in range(len(bin_edges)-1)]

    results = {}
    for distribution in MY_DISTRIBUTIONS:
        # Отримуємо параметри розподілу
        params = distribution.fit(data)
        
        # Виділяємо потрібні нам частини параметрів
        arg = params[:-2]
        loc = params[-2]
        scale = params[-1]
    
        # Розраховуємо підігнаний PDF і похибку підгонки розподілу
        pdf_values = [distribution.pdf(c, loc=loc, scale=scale, *arg) for c in central_values]
        
        # Рахуємо суму квадратів помилки
        sse = np.sum(np.power(frequencies - pdf_values, 2.0))
        
        # Збираємо результати
        results[distribution] = [sse, arg, loc, scale]
        
    results = {k: results[k] for k in sorted(results, key=results.get)}
    return results

Набір даних, який будемо досліджувати в цьому завданні, містить океанографічні та надземні метеорологічні дані, отримані з низки буїв, розташованих по всій екваторіальній частині Тихого океану:
https://archive.ics.uci.edu/ml/datasets/El+Nino  

Дані зібрано за кілька років, колонки - це місяці. Тому перед аналізом нам треба трансформувати набір даних.

In [ ]:
df = sm.datasets.elnino.load_pandas().data.set_index('YEAR')

In [ ]:
df

In [ ]:
data = pd.Series(df.values.ravel()) # df.ravel() - вертає плоский масив даних

In [ ]:
results = fit_data(data)

In [ ]:
st.beta

In [ ]:
plot_histogram(data, results, 5)

# Z-test

In [ ]:
from statsmodels.stats.weightstats import ztest  

## На одній вибірці

Згенеруємо випадковий масив із 50 чисел, що мають середнє значення 110 і стандартне відхилення 15, аналогічно до даних IQ, які ми припускаємо в задачі.

In [ ]:
mean_iq = 110
sd_iq = 15 / math.sqrt(50)
alpha = 0.05
null_mean = 100

In [ ]:
# генеруємо дані
data = np.random.randn(50)*15 + mean_iq

In [ ]:
sd_iq

In [ ]:
print('mean=%.2f stdv=%.2f' % (np.mean(data), np.std(data)))
  

In [ ]:
data

Тепер проводимо тест. У цій функції ми передали дані, у параметрі значення ми передали середнє значення в нульовій гіпотезі, в альтернативній гіпотезі ми перевіряємо, чи більше середнє значення

In [ ]:
ztest_Score, p_value = ztest(data, value = null_mean, alternative='larger')

In [ ]:
ztest_Score, p_value

Функція виводить p_value і z-score, що відповідають цьому значенню, ми порівнюємо p-значення з альфа, якщо воно більше альфа, то ми не приймаємо нульову гіпотезу, інакше ми її відхиляємо.

In [ ]:
if(p_value <  alpha):
    print("Відхилити Н0.")
else:
    print("Н0 не може бути відхилена.")

## На двох вибірках

Порівняємо рівні IQ у двох різних містах.

In [ ]:
cityA = [82, 84, 85, 89, 91, 91, 92, 94, 99, 99,
         105, 109, 109, 109, 110, 112, 112, 113, 114, 114]

cityB = [90, 91, 91, 91, 95, 95, 99, 99, 108, 109,
         109, 114, 115, 116, 117, 117, 128, 129, 130, 133]

In [ ]:
np.mean(cityA), np.mean(cityB)

Виконуємо тест.

Важливий параметр методу ztest:
- value : float  
    In the one sample case, value is the mean of x1 under the Null
    hypothesis.
    In the two sample case, value is the difference between mean of x1 and
    mean of x2 under the Null hypothesis. The test statistic is
    `x1_mean - x2_mean - value`.
    
Метод z-test вертає

- tstat : float,
    test statistic
- pvalue : float,
    pvalue of the t-test

In [ ]:
ztest(cityA, cityB, value=0)

Статистика для двох вибірок z-критерію становить -1.9953, а відповідне p-value дорівнює 0.0460.

Оскільки p-value < 0.05, у нас є достатньо доказів, щоб відкинути нульову гіпотезу. Іншими словами, середній рівень IQ значно різниться між двома містами.